## Setup

### Configure API keys

To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

### Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLO11 via Ultralytics

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

## Fine-tune YOLO11 on Custom Data

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. In this tutorial, we will use one of the [datasets](https://universe.roboflow.com/liangdianzhong/-qvdww) available on [Roboflow Universe](https://universe.roboflow.com/). When downloading, make sure to select the `yolov11` export format.

In [ ]:
from dotenv import load_dotenv
import os

HOME = os.getcwd()

#env_path = r"C:\Users\SABIO\Documents\GitHub\Pool-Detection-CNN\.env"
env_path = f"{HOME}/.env"

load_dotenv(dotenv_path=env_path)
robo_token = os.getenv("ROBOFLOW_API_KEY")

!mkdir {HOME}/datasets
%cd {HOME}/datasets

#from google.colab import userdata
from roboflow import Roboflow



from roboflow import Roboflow
rf = Roboflow(api_key=robo_token)
project = rf.workspace("spencer-kmi6v").project("screen-snooper-detection")
version = project.version(2)
dataset = version.download("yolov11", location='../datasets/YOLO')

## Custom Training

Run the following block to train the model. Keep in mind it is currently set at 100 epochs which depending on your gpu, might take some time.

In [ ]:
#%cd {HOME}
import os

HOME = r"C:\Users\SABIO\Documents\GitHub\Pool-Detection-CNN\references"
os.chdir(HOME)

!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 plots=True project='../models' name='detect/train'

## Validate Fine-Tuned Model

Now we will validate the fine-tuned model. Here it might make sense to test some confidence thresholds. You can add `conf=0.25` or `conf=0.5` to see how it affects your results.

In [ ]:
!yolo task=detect mode=val model=../models/detect/train/weights/best.pt data={dataset.location}/data.yaml project=../models name=detect/val

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'../models/detect/val/confusion_matrix.png', width=600)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'../models/detect/train/results.png', width=600)

## Inference with Custom model

Here I added `conf=0.5` as my model was detecting some false positives (pools where there were no pools) and the results had improved.

In [ ]:
!yolo task=detect mode=predict model=../models/detect/train/weights/best.pt conf=0.5 source={dataset.location}/test/images save=True project=../models name=detect/predict

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
import os
from IPython.display import Image as IPyImage, display

latest_folder = max(glob.glob(f'../models/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[:3]:
    display(IPyImage(filename=img, width=600))
    print("\n")

## Deploy model on Roboflow

Once you have finished training your YOLOv11 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv11 weights.

In [ ]:
project.version(dataset.version).deploy(model_type="yolov11", model_path=f"../models/detect/train/")

In [ ]:
!pip install inference

In [ ]:
import os, random, cv2
import supervision as sv
import IPython
import inference

model_id = project.id.split("/")[1] + "/" + dataset.version
model = inference.get_model(model_id, robo_token)

# Location of test set images
test_set_loc = dataset.location + f"/test/images/"
test_images = os.listdir(test_set_loc)

# Run inference on 4 random test images, or fewer if fewer images are available
for img_name in random.sample(test_images, min(4, len(test_images))):
    print("Running inference on " + img_name)

    # Load image
    image = cv2.imread(os.path.join(test_set_loc, img_name))

    # Perform inference
    results = model.infer(image, confidence=0.4, overlap=30)[0]
    detections = sv.Detections.from_inference(results)

    # Annotate boxes and labels
    box_annotator = sv.BoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    annotated_image = box_annotator.annotate(scene=image, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

    # Display annotated image
    _, ret = cv2.imencode('.jpg', annotated_image)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)
